In [2]:
import os
import pandas as pd
# requires dammit env
# source activate dammit
from dammit.fileio.gff3 import GFF3Parser

# Make list of genes being quantified

Salmon was run with the de novo transcriptome assemblies, with the original Trinity ID in the header for each fasta contig record, like this (example salmon output file):

```
Name	Length	EffectiveLength	TPM	NumReads
TRINITY_DN2202_c0_g1_i1	210	39.818	2.683835	2.000000
TRINITY_DN2270_c0_g1_i1	213	41.064	0.000000	0.000000
TRINITY_DN2201_c0_g1_i1	266	69.681	0.766816	1.000000
TRINITY_DN2222_c0_g1_i1	243	55.794	2.873014	3.000000
TRINITY_DN2291_c0_g1_i1	245	56.916	0.000000	0.000000
TRINITY_DN2269_c0_g1_i1	294	89.251	0.000000	0.000000
TRINITY_DN2269_c1_g1_i1	246	57.479	0.000000	0.000000
TRINITY_DN2279_c0_g1_i1	426	207.443	0.000000	0.000000
TRINITY_DN2262_c0_g1_i1	500	280.803	0.190459	1.000912
TRINITY_DN2253_c0_g1_i1	1523	1303.116	0.164015	4.000000
TRINITY_DN2287_c0_g1_i1	467	247.962	0.000000	0.000000
TRINITY_DN2287_c1_g1_i1	325	113.826	0.469425	1.000000
TRINITY_DN2237_c0_g1_i1	306	98.441	0.542788	1.000000
TRINITY_DN2237_c0_g2_i1	307	99.229	0.000000	0.000000
TRINITY_DN2250_c0_g1_i1	368	151.832	0.000000	0.000000
TRINITY_DN2250_c1_g1_i1	271	72.988	0.000000	0.000000
TRINITY_DN2208_c0_g1_i1	379	162.080	1.978014	6.000000
TRINITY_DN2277_c0_g1_i1	269	71.657	0.745677	1.000000
TRINITY_DN2231_c0_g1_i1	209	39.409	0.000000	0.000000
TRINITY_DN2231_c1_g1_i1	334	121.411	0.000000	0.000000
TRINITY_DN2204_c0_g1_i1	287	84.121	0.000000	0.000000
```

Takes each salmon output .quant file (like the one above), parse the "i" (isoform) off of the ID to get a list of genes and isoforms identified by Trinity.

In [3]:
species_dirs = "/Users/johnsolk/Documents/UCDavis/Whitehead/kfish_salmon/kfish_salmon_v0.14.1/"
out_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/counts_collapsed_July2019/"
dirs = os.listdir(species_dirs)
for species in dirs:
    if species != ".DS_Store":
        count = 0
        species_gene_file = out_dir + species + "_contig_gene.csv"
        files = os.listdir(species_dirs + "/" + species)
        for quant_dir in files:
            if quant_dir != ".DS_Store":
                while count == 0:
                    replicate = quant_dir.split(".")[0]
                    print(replicate)
                    quant_files = os.listdir(species_dirs + "/" + species + "/" + quant_dir)
                    for file in quant_files:
                        if file.endswith(".sf"):
                            table = pd.read_csv(species_dirs + "/" + species + "/" + quant_dir + "/" + file,sep="\t")
                            print("Contigs:",table.shape)
                            contig_names = pd.DataFrame(table.Name.str.split("_",5).tolist(),columns = ['Trinity','Contig','Component','Gene','Isoform'])
                            contig_names2 = pd.DataFrame()
                            contig_names2['Name'] = contig_names[['Trinity','Contig','Component','Gene','Isoform']].apply(lambda x: '_'.join(x),axis=1)
                            contig_names2['Gene'] = contig_names[['Trinity','Contig','Component','Gene']].apply(lambda x: '_'.join(x),axis=1)
                            print("Trinity 'genes':",contig_names2['Gene'].nunique())
                            contig_names2.to_csv(species_gene_file)
                            print("File written:",species_gene_file)
                            count += 1

L_goodei_transfer_2
Contigs: (320120, 5)
Trinity 'genes': 196580
File written: /Users/johnsolk/Documents/UCDavis/Whitehead/counts_collapsed_July2019/L_goodei_contig_gene.csv
F_notti_FW_2
Contigs: (131628, 5)
Trinity 'genes': 90275
File written: /Users/johnsolk/Documents/UCDavis/Whitehead/counts_collapsed_July2019/F_nottii_contig_gene.csv
L_parva_FW_3
Contigs: (341080, 5)
Trinity 'genes': 204418
File written: /Users/johnsolk/Documents/UCDavis/Whitehead/counts_collapsed_July2019/L_parva_contig_gene.csv
F_similis_transfer_1
Contigs: (426262, 5)
Trinity 'genes': 296375
File written: /Users/johnsolk/Documents/UCDavis/Whitehead/counts_collapsed_July2019/F_similis_contig_gene.csv
F_notatus_FW_2
Contigs: (343128, 5)
Trinity 'genes': 211914
File written: /Users/johnsolk/Documents/UCDavis/Whitehead/counts_collapsed_July2019/F_notatus_contig_gene.csv
F_heteroclitusMDPP_FW_3
Contigs: (552204, 5)
Trinity 'genes': 369971
File written: /Users/johnsolk/Documents/UCDavis/Whitehead/counts_collapsed_July

# Using dammit's `GFF3Parser` function 
1. Digests the gff3 file for each species (output from dammit, downloaded from the farm cluster)
2. Sorts each contig by E-value score
3. Assigns the lowest E-value score for each contig 
4. Separately, for each contig, saves gene names from the NCBI annotated F. heteroclitus genome

In [20]:
annotations_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/gff_annotations/"
conversion_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/kfish_namemaps/kfish_namemaps/"
annotations = os.listdir(annotations_dir)
conversions = os.listdir(conversion_dir)
for annotation in annotations:
    if annotation != ".DS_Store":
        species = annotation.split(".")[0]
        if species == "F_olivaceous":
            species = "F_olivaceus"
        if species == "F_notti":
            species = "F_nottii"
        print(species)
        print(annotation)
        name = annotations_dir + annotation
        annotations = GFF3Parser(filename=name).read()
        annotations = annotations.dropna(subset=['Name'])
        annotations["length"] = annotations["end"].subtract(annotations["start"], fill_value=0)
        pickonename = annotations.sort_values(by=['seqid', 'score'], ascending=True).query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name','Note','database','Dbxref','start','end','length']]
        pickonename = pickonename.dropna(axis=0,how="all")
        print('Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):')
        print(pickonename.shape)
        #contigs_w_expression = pd.read_csv(out_dir + species + '_trinity_contig_gene_table_filtered.txt',sep="\t",names = ['contig','gene'])
        #contigs = species + "_trinity_contig_gene_table.txt"
        #contigs_table = pd.read_csv(out_dir + contigs,sep="\t",names = ['contig','gene'])
        #print('Number of contigs:')
        #print(contigs_table.shape)
        conversion_file = [i for i in conversions if i.startswith(species)][0]
        conversion = pd.read_csv(conversion_dir + conversion_file)
        conversion['contig'], conversion['info'] = conversion['original'].str.split(' ', 1).str
        conversion = conversion[['contig','renamed']]
        print("Number of contigs")
        print(conversion.shape)
        #contigs_w_expression_conversion = pd.merge(conversion, contigs_w_expression, on="contig")
        #contigs_w_expression_conversion.columns = ['contig','seqid','gene']
        annotations_expression = pd.merge(annotations,pickonename,on='seqid')
        print("Number of unique gene names:")
        print(len(annotations_expression.Name_y.unique()))

F_heteroclitusMDPP
F_heteroclitusMDPP.trinity_out.Trinity.fasta.dammit.fixed.gff3
Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(177922, 8)
Number of contigs
(552205, 2)
Number of unique gene names:
66472
F_chrysotus
F_chrysotus.trinity_out.Trinity.fasta.dammit.fixed.gff3
Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(149463, 8)
Number of contigs
(328866, 2)
Number of unique gene names:
54340
F_catanatus
F_catanatus.trinity_out.Trinity.fasta.dammit.fixed.gff3
Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(145243, 8)
Number of contigs
(325956, 2)
Number of unique gene names:
56122
F_parvapinis
F_parvapinis.trinity_out.Trinity.fasta.dammit.fixed.gff3
Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(117505, 8)
Number of contigs
(298549, 2)
Num

In [18]:
print(conversions)

['F_diaphanus.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_chrysotus.trinity_out.Trinity.fasta.dammit.namemap.csv', 'L_goodei.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_heteroclitusMDPL.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_grandis.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_olivaceous.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_zebrinus.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_catanatus.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_sciadicus.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_heteroclitusMDPP.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_similis.trinity_out.Trinity.fasta.dammit.namemap.csv', 'A_xenica.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_parvapinis.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_notatus.trinity_out.Trinity.fasta.dammit.namemap.csv', 'L_parva.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_notti.trinity_out.Trinity.fasta.dammit.namemap.csv', 'F_rathbuni.trinity_out.Trinity.fasta.dammit

# *Fundulus heteroclitus* gene models only

In [23]:
#RefSeq_NCBI_annotations = GFF3Parser(filename="/Users/johnsolk/Documents/UCDavis/Whitehead/tmp_Fhet_files/ref_Fundulus_heteroclitus-3.0.2_top_level.gff3").read()
RefSeq_NCBI_annotations = GFF3Parser(filename="/Users/johnsolk/Documents/UCDavis/Whitehead/ref_Fundulus_heteroclitus-3.0.2_scaffolds.gff3").read()

/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):


In [24]:
RefSeq_NCBI_annotations.head()

Dbxref  Gap     ID Is_circular  \
0                               taxon:8078  NaN    id0         NaN   
1                         GeneID:105920463  NaN  gene0         NaN   
2  GeneID:105920463,Genbank:XM_012856390.1  NaN   rna0         NaN   
3  GeneID:105920463,Genbank:XM_012856390.1  NaN    id1         NaN   
4  GeneID:105920463,Genbank:XM_012856390.1  NaN    id2         NaN   

             Name Note Parent Target anticodon chromosome   ...    \
0         Unknown  NaN    NaN    NaN       NaN    Unknown   ...     
1    LOC105920463  NaN    NaN    NaN       NaN        NaN   ...     
2  XM_012856390.1  NaN  gene0    NaN       NaN        NaN   ...     
3             NaN  NaN   rna0    NaN       NaN        NaN   ...     
4             NaN  NaN   rna0    NaN       NaN        NaN   ...     

            seqid     sex  source  start start_range strand   transcript_id  \
0  NW_012224401.1  female  RefSeq      0         NaN      +             NaN   
1  NW_012224401.1     NaN  Gnomon   5034         NaN      -             NaN   
2  NW_012224401.1     NaN  Gnomon   5034         NaN      -  XM_012856390.1   
3  NW_012224401.1     NaN  Gnomon  10044         NaN      -  XM_012856390.1   
4  NW_012224401.1     NaN  Gnomon   8334         NaN      -  XM_012856390.1   

  transl_except transl_table    type  
0           NaN          NaN  region  
1           NaN          NaN    gene  
2           NaN          NaN    mRNA  
3           NaN          NaN    exon  
4           NaN          NaN    exon  

[5 rows x 50 columns]

In [25]:
# Convert protein ID, e.g. "XP_" to gene ID
# how many gene models?
RefSeq_NCBI_genenames = RefSeq_NCBI_annotations.dropna(subset=['seqid','Dbxref','transcript_id']).drop_duplicates(subset='Dbxref')[['seqid','Dbxref','transcript_id']]

In [26]:
print(RefSeq_NCBI_genenames.shape)

(35873, 3)


In [27]:
print(RefSeq_NCBI_genenames.head())

              seqid                                   Dbxref   transcript_id
2    NW_012224401.1  GeneID:105920463,Genbank:XM_012856390.1  XM_012856390.1
10   NW_012224401.1  GeneID:105915213,Genbank:XM_012853081.1  XM_012853081.1
54   NW_012224401.1  GeneID:105915271,Genbank:XM_012849362.1  XM_012849362.1
77   NW_012224401.1  GeneID:105915271,Genbank:XM_012849437.1  XM_012849437.1
143  NW_012224401.1  GeneID:105915419,Genbank:XM_012849553.1  XM_012849553.1


In [28]:
RefSeq_NCBI_genenames['split_1'], RefSeq_NCBI_genenames['split_2'], RefSeq_NCBI_genenames['split_3'] = RefSeq_NCBI_genenames['Dbxref'].str.split(':',0).str

In [29]:
print(RefSeq_NCBI_genenames.head())

              seqid                                   Dbxref   transcript_id  \
2    NW_012224401.1  GeneID:105920463,Genbank:XM_012856390.1  XM_012856390.1   
10   NW_012224401.1  GeneID:105915213,Genbank:XM_012853081.1  XM_012853081.1   
54   NW_012224401.1  GeneID:105915271,Genbank:XM_012849362.1  XM_012849362.1   
77   NW_012224401.1  GeneID:105915271,Genbank:XM_012849437.1  XM_012849437.1   
143  NW_012224401.1  GeneID:105915419,Genbank:XM_012849553.1  XM_012849553.1   

    split_1            split_2         split_3  
2    GeneID  105920463,Genbank  XM_012856390.1  
10   GeneID  105915213,Genbank  XM_012853081.1  
54   GeneID  105915271,Genbank  XM_012849362.1  
77   GeneID  105915271,Genbank  XM_012849437.1  
143  GeneID  105915419,Genbank  XM_012849553.1  


In [30]:
RefSeq_NCBI_genenames_sub = RefSeq_NCBI_genenames[(RefSeq_NCBI_genenames["split_1"] == 'GeneID')]
print(RefSeq_NCBI_genenames_sub.head())

              seqid                                   Dbxref   transcript_id  \
2    NW_012224401.1  GeneID:105920463,Genbank:XM_012856390.1  XM_012856390.1   
10   NW_012224401.1  GeneID:105915213,Genbank:XM_012853081.1  XM_012853081.1   
54   NW_012224401.1  GeneID:105915271,Genbank:XM_012849362.1  XM_012849362.1   
77   NW_012224401.1  GeneID:105915271,Genbank:XM_012849437.1  XM_012849437.1   
143  NW_012224401.1  GeneID:105915419,Genbank:XM_012849553.1  XM_012849553.1   

    split_1            split_2         split_3  
2    GeneID  105920463,Genbank  XM_012856390.1  
10   GeneID  105915213,Genbank  XM_012853081.1  
54   GeneID  105915271,Genbank  XM_012849362.1  
77   GeneID  105915271,Genbank  XM_012849437.1  
143  GeneID  105915419,Genbank  XM_012849553.1  


In [31]:
RefSeq_NCBI_genenames_sub['GeneID'], RefSeq_NCBI_genenames_sub['Genbank'] = RefSeq_NCBI_genenames_sub['split_2'].str.split(',', 0).str

In [32]:
print(RefSeq_NCBI_genenames_sub.head())

              seqid                                   Dbxref   transcript_id  \
2    NW_012224401.1  GeneID:105920463,Genbank:XM_012856390.1  XM_012856390.1   
10   NW_012224401.1  GeneID:105915213,Genbank:XM_012853081.1  XM_012853081.1   
54   NW_012224401.1  GeneID:105915271,Genbank:XM_012849362.1  XM_012849362.1   
77   NW_012224401.1  GeneID:105915271,Genbank:XM_012849437.1  XM_012849437.1   
143  NW_012224401.1  GeneID:105915419,Genbank:XM_012849553.1  XM_012849553.1   

    split_1            split_2         split_3     GeneID  Genbank  
2    GeneID  105920463,Genbank  XM_012856390.1  105920463  Genbank  
10   GeneID  105915213,Genbank  XM_012853081.1  105915213  Genbank  
54   GeneID  105915271,Genbank  XM_012849362.1  105915271  Genbank  
77   GeneID  105915271,Genbank  XM_012849437.1  105915271  Genbank  
143  GeneID  105915419,Genbank  XM_012849553.1  105915419  Genbank  


In [33]:
geneID_unique_scaffold = RefSeq_NCBI_genenames_sub.dropna(subset=['GeneID','seqid','transcript_id']).drop_duplicates(subset='seqid')[['GeneID','seqid','transcript_id']]
print(geneID_unique_scaffold.head())
print(geneID_unique_scaffold.shape)

          GeneID           seqid   transcript_id
2      105920463  NW_012224401.1  XM_012856390.1
5463   105932240  NW_012224402.1  XM_012871340.1
10844  105915837  NW_012224403.1  XM_012850083.1
14351  105916749  NW_012224404.1  XM_012851366.1
18439  105917961  NW_012224405.1  XM_012852925.1
(3121, 3)


In [34]:
geneID = RefSeq_NCBI_genenames_sub.dropna(subset=['GeneID','seqid','transcript_id']).drop_duplicates(subset='GeneID')[['GeneID','seqid','transcript_id']]
print(geneID.head())
print(geneID.shape)

        GeneID           seqid   transcript_id
2    105920463  NW_012224401.1  XM_012856390.1
10   105915213  NW_012224401.1  XM_012853081.1
54   105915271  NW_012224401.1  XM_012849362.1
143  105915419  NW_012224401.1  XM_012849553.1
186  105915503  NW_012224401.1  XM_012849668.1
(24862, 3)


In [35]:
transcript_ID = RefSeq_NCBI_genenames_sub.dropna(subset=['GeneID','seqid','transcript_id']).drop_duplicates(subset='transcript_id')[['GeneID','seqid','transcript_id']]
print(transcript_ID.head())
print(transcript_ID.shape)

        GeneID           seqid   transcript_id
2    105920463  NW_012224401.1  XM_012856390.1
10   105915213  NW_012224401.1  XM_012853081.1
54   105915271  NW_012224401.1  XM_012849362.1
77   105915271  NW_012224401.1  XM_012849437.1
143  105915419  NW_012224401.1  XM_012849553.1
(35873, 3)


In [36]:
protein_ID = RefSeq_NCBI_annotations.dropna(subset=['protein_id','seqid','product','gene']).drop_duplicates(subset='protein_id')[['protein_id','seqid','product','gene']]
print(protein_ID.head())
print(protein_ID.shape)
protein_ID.to_csv("~/Documents/UCDavis/Whitehead/NCBI_protein_ID.csv")

         protein_id           seqid  \
6    XP_012711844.1  NW_012224401.1   
32   XP_012708535.1  NW_012224401.1   
100  XP_012704816.1  NW_012224401.1   
121  XP_012704891.1  NW_012224401.1   
165  XP_012705007.1  NW_012224401.1   

                                             product          gene  
6                        homeobox protein CDX-1-like  LOC105920463  
32      platelet-derived growth factor receptor beta        pdgfrb  
100  macrophage colony-stimulating factor 1 receptor         csf1r  
121  macrophage colony-stimulating factor 1 receptor         csf1r  
165                  HMG domain-containing protein 3        hmgxb3  
(33705, 4)


In [17]:
#merge protein ID and transcript ID
protein_transcript = pd.merge(protein_ID,transcript_ID,how="inner",on="seqid").dropna()
print(protein_transcript.shape)
print(protein_transcript.head())

(3042359, 6)
       protein_id           seqid                 product  gene     GeneID  \
0  XP_012711844.1  NW_012224401.1  homeobox protein CDX-1  cdx1  105920463   
1  XP_012711844.1  NW_012224401.1  homeobox protein CDX-1  cdx1  105915213   
2  XP_012711844.1  NW_012224401.1  homeobox protein CDX-1  cdx1  105915271   
3  XP_012711844.1  NW_012224401.1  homeobox protein CDX-1  cdx1  105915271   
4  XP_012711844.1  NW_012224401.1  homeobox protein CDX-1  cdx1  105915419   

    transcript_id  
0  XM_012856390.2  
1  XM_021321288.1  
2  XM_012849362.2  
3  XM_012849437.2  
4  XM_021323826.1  


In [18]:
print("Number of unique scaffold ID ('NW_') :",len(pd.unique(RefSeq_NCBI_annotations['seqid'].dropna())))
print("Number of unique transcripts ('XM_') :",len(pd.unique(RefSeq_NCBI_annotations['transcript_id'].dropna())))
print("Number of unique protein ID ('XP_') :",len(pd.unique(RefSeq_NCBI_annotations['protein_id'].dropna())))
print("Number of unique gene ID (parsed from 'Dbxref') :",len(geneID))

Number of unique scaffold ID ('NW_') : 10180
Number of unique transcripts ('XM_') : 41170
Number of unique protein ID ('XP_') : 38234
Number of unique gene ID (parsed from 'Dbxref') : 25600


In [19]:
RefSeq_NCBI_annotations.head()

Dbxref  Gap     ID Is_circular  \
0                               taxon:8078  NaN    id0         NaN   
1                         GeneID:105920463  NaN  gene0         NaN   
2  GeneID:105920463,Genbank:XM_012856390.2  NaN   rna0         NaN   
3  GeneID:105920463,Genbank:XM_012856390.2  NaN    id1         NaN   
4  GeneID:105920463,Genbank:XM_012856390.2  NaN    id2         NaN   

             Name Note Parent Target anticodon chromosome        ...         \
0         Unknown  NaN    NaN    NaN       NaN    Unknown        ...          
1            cdx1  NaN    NaN    NaN       NaN        NaN        ...          
2  XM_012856390.2  NaN  gene0    NaN       NaN        NaN        ...          
3             NaN  NaN   rna0    NaN       NaN        NaN        ...          
4             NaN  NaN   rna0    NaN       NaN        NaN        ...          

  splices standard_name  start start_range  strand   transcript_id  \
0     NaN           NaN      0         NaN       +             NaN   
1     NaN           NaN   5029         NaN       -             NaN   
2     NaN           NaN   5029         NaN       -  XM_012856390.2   
3     NaN           NaN  10044         NaN       -  XM_012856390.2   
4     NaN           NaN   8334         NaN       -  XM_012856390.2   

  transl_except transl_table    type weighted_identity  
0           NaN          NaN  region               NaN  
1           NaN          NaN    gene               NaN  
2           NaN          NaN    mRNA               NaN  
3           NaN          NaN    exon               NaN  
4           NaN          NaN    exon               NaN  

[5 rows x 61 columns]

## Merge NCBI annotations with contigs annotated to Fhet

In [161]:
# print number of Fhet "genes" in each:
Fhet_counts_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/Fhet_counts/"
Fhet_counts_ncbi = "/Users/johnsolk/Documents/UCDavis/Whitehead/Fhet_counts_ncbi/"
files = os.listdir(Fhet_counts_dir)
print(files)
for table_file in files:
    if table_file.endswith(".csv"):
        table = pd.read_csv(Fhet_counts_dir + table_file)
        print("Contigs (one Fhet gene selected, best E-value < 1E-05 match per contig):",table.groupby('seqid').count().shape)
        print("Unique Fhet genes:",table.groupby('Name').count().shape)
        table['split_1'], table['split_2'], table['split_3'],table['split_4'],table['split_5'] = table['Name'].str.split('|',0).str
        table.columns = ['index','seqid', 'Name', 'start', 'end', 'length', 'split_1', 'id','split_3', 'protein_id', 'Notes']
        table = table[['seqid', 'Name', 'start', 'end', 'length', 'id','protein_id', 'Notes']]
        table = pd.merge(table,protein_transcript,how="inner",on="protein_id")
        print(table.head())
        table = table.drop_duplicates(subset='seqid_x')[['seqid_x', 'Name', 'start', 'end', 'length', 'id','protein_id', 'Notes','seqid_y','product','gene','GeneID']]
        # there are multiple NCBI GeneID and tarnscript_id per seqid (Trinity transcript)
        # Yet, it's the same protein...
        # Meaning, the Fhet reference is fragemented
        table = table[['seqid_x', 'start', 'end', 'length', 'id','protein_id', 'Notes','seqid_y','product','gene','GeneID']]
        table.columns = ['seqid', 'start', 'end', 'length', 'NCBI.id','NCBI.protein', 'Notes','NCBI.trasncript','NCBI.product','gene','NCBI.gene']
        conversion_file = [i for i in conversions if i.startswith(species)][0]
        conversion = pd.read_csv(conversion_dir + conversion_file)
        conversion['contig'], conversion['info'] = conversion['original'].str.split(' ', 1).str
        conversion = conversion[['contig','renamed']]
        conversion.columns = ['contig','seqid']
        table = pd.merge(conversion, table, on="seqid")
        print(table.head())
        print(table.shape)
        file_out = Fhet_counts_ncbi + table_file + "_NCBI.csv"
        table.to_csv(file_out)
        

['F_catanatus_genenames.csv', 'F_chrysotus_genenames.csv', 'F_notatus_genenames.csv', 'F_olivaceus_genenames.csv', 'L_goodei_genenames.csv', 'F_nottii_genenames.csv', '.DS_Store', 'F_grandis_genenames.csv', 'F_diaphanus_genenames.csv', 'A_xenica_genenames.csv', 'F_parvapinis_genenames.csv', 'F_rathbuni_genenames.csv', 'F_similis_genenames.csv', 'F_heteroclitusMDPL_genenames.csv', 'L_parva_genenames.csv', 'F_zebrinus_genenames.csv', 'F_sciadicus_genenames.csv', 'F_heteroclitusMDPP_genenames.csv']
Contigs (one Fhet gene selected, best E-value < 1E-05 match per contig): (78538, 5)
Unique Fhet genes: (22435, 5)
             seqid_x                                               Name  \
0  Transcript_100057  gi|831528406|ref|XP_012720105.1| PREDICTED: E3...   
1  Transcript_100057  gi|831528406|ref|XP_012720105.1| PREDICTED: E3...   
2  Transcript_100057  gi|831528406|ref|XP_012720105.1| PREDICTED: E3...   
3  Transcript_100057  gi|831528406|ref|XP_012720105.1| PREDICTED: E3...   
4  Transcr

                     contig          seqid  start  end  length    NCBI.id  \
0  TRINITY_DN73275_c0_g2_i1   Transcript_2     30  133     103  831573055   
1  TRINITY_DN73287_c0_g1_i1   Transcript_3     30  180     150  831573058   
2  TRINITY_DN73227_c0_g1_i1   Transcript_4     30  138     108  831573058   
3  TRINITY_DN73260_c0_g1_i1   Transcript_5     30  148     118  831573058   
4  TRINITY_DN73242_c0_g2_i1  Transcript_15      0   68      68  831544836   

     NCBI.protein                                              Notes  \
0  XP_012735565.1   PREDICTED: bcl-2-modifying factor isoform X1 ...   
1  XP_012735566.1   PREDICTED: bcl-2-modifying factor isoform X2 ...   
2  XP_012735566.1   PREDICTED: bcl-2-modifying factor isoform X2 ...   
3  XP_012735566.1   PREDICTED: bcl-2-modifying factor isoform X2 ...   
4  XP_012726018.1   PREDICTED: probable G-protein coupled recepto...   

  NCBI.trasncript                            NCBI.product          gene  \
0  NW_012234514.1       bcl-2

                     contig          seqid  start  end  length    NCBI.id  \
0  TRINITY_DN73240_c0_g1_i1  Transcript_16     37  355     318  831482748   
1  TRINITY_DN32561_c0_g1_i1  Transcript_34    327  557     230  831543115   
2  TRINITY_DN32542_c0_g1_i1  Transcript_43     54  376     322  831523103   
3  TRINITY_DN32516_c0_g1_i1  Transcript_46     20  400     380  831580562   
4  TRINITY_DN32505_c0_g1_i1  Transcript_47    131  750     619  831580562   

     NCBI.protein                                              Notes  \
0  XP_012732171.1   PREDICTED: deoxyribose-phosphate aldolase [Fu...   
1  XP_012725411.1   PREDICTED: ephrin-A5 isoform X1 [Fundulus het...   
2  XP_012718222.1   PREDICTED: leucine-rich repeat-containing pro...   
3  XP_012738220.1       PREDICTED: plastin-2 [Fundulus heteroclitus]   
4  XP_012738220.1       PREDICTED: plastin-2 [Fundulus heteroclitus]   

  NCBI.trasncript                               NCBI.product    gene  \
0  NW_012224481.1             de

                     contig           seqid  start  end  length    NCBI.id  \
0  TRINITY_DN73209_c0_g1_i1   Transcript_26      0   66      66  831534962   
1  TRINITY_DN73217_c0_g1_i1   Transcript_27      0   51      51  831534962   
2  TRINITY_DN32550_c0_g1_i1   Transcript_49      0   21      21  831522803   
3  TRINITY_DN81321_c1_g1_i1  Transcript_107      1   91      90  831493469   
4  TRINITY_DN81390_c0_g1_i1  Transcript_113     18  125     107  831473992   

     NCBI.protein                                              Notes  \
0  XP_012722439.1   PREDICTED: basement membrane-specific heparan...   
1  XP_012722439.1   PREDICTED: basement membrane-specific heparan...   
2  XP_012718119.1   PREDICTED: chromobox protein homolog 3-like i...   
3  XP_012707969.1   PREDICTED: cAMP-specific 3',5'-cyclic phospho...   
4  XP_012717323.1   PREDICTED: glutamate receptor 3 isoform X3 [F...   

  NCBI.trasncript                                       NCBI.product  \
0  NW_012234307.1  basemen

Contigs (one Fhet gene selected, best E-value < 1E-05 match per contig): (74555, 5)
Unique Fhet genes: (22338, 5)
             seqid_x                                               Name  \
0  Transcript_100000  gi|831539324|ref|XP_012723974.1| PREDICTED: ad...   
1  Transcript_100000  gi|831539324|ref|XP_012723974.1| PREDICTED: ad...   
2  Transcript_100000  gi|831539324|ref|XP_012723974.1| PREDICTED: ad...   
3  Transcript_100000  gi|831539324|ref|XP_012723974.1| PREDICTED: ad...   
4  Transcript_100000  gi|831539324|ref|XP_012723974.1| PREDICTED: ad...   

   start  end  length         id      protein_id  \
0     73  132      59  831539324  XP_012723974.1   
1     73  132      59  831539324  XP_012723974.1   
2     73  132      59  831539324  XP_012723974.1   
3     73  132      59  831539324  XP_012723974.1   
4     73  132      59  831539324  XP_012723974.1   

                                               Notes         seqid_y  \
0   PREDICTED: adenylate cyclase type 5 isoform X.

                     contig           seqid  start  end  length    NCBI.id  \
0  TRINITY_DN32550_c0_g1_i1   Transcript_49      1  206     205  831545408   
1  TRINITY_DN32550_c0_g2_i1   Transcript_50      0  172     172  831538787   
2  TRINITY_DN32577_c0_g1_i1   Transcript_63      0   55      55  831503985   
3  TRINITY_DN67518_c1_g1_i1  Transcript_357      0  138     138  831528929   
4  TRINITY_DN87446_c0_g1_i1  Transcript_407     47  211     164  831520549   

     NCBI.protein                                              Notes  \
0  XP_012726222.1   PREDICTED: photoreceptor-specific nuclear rec...   
1  XP_012723788.1   PREDICTED: protocadherin-8 [Fundulus heterocl...   
2  XP_012711686.1   PREDICTED: solute carrier family 13 member 5-...   
3  XP_012720290.1   PREDICTED: choline transporter-like protein 5...   
4  XP_012717313.1   PREDICTED: UDP-N-acetylglucosamine transferas...   

  NCBI.trasncript                                       NCBI.product  \
0  NW_012234349.1       ph

                     contig           seqid  start  end  length    NCBI.id  \
0  TRINITY_DN81334_c0_g1_i1   Transcript_95     28   90      62  831497731   
1  TRINITY_DN56987_c0_g1_i1  Transcript_166      0   72      72  831491849   
2  TRINITY_DN48616_c0_g1_i1  Transcript_233     62   86      24  831484829   
3  TRINITY_DN33983_c0_g1_i1  Transcript_586      5   40      35  831576519   
4  TRINITY_DN33983_c0_g2_i1  Transcript_587      0   23      23  831576519   

     NCBI.protein                                              Notes  \
0  XP_012709482.1   PREDICTED: nuclear receptor ROR-alpha-like is...   
1  XP_012707401.1   PREDICTED: protein delta homolog 1 [Fundulus ...   
2  XP_012704828.1   PREDICTED: homeobox protein goosecoid-2 [Fund...   
3  XP_012736794.1   PREDICTED: uncharacterized protein LOC1059392...   
4  XP_012736794.1   PREDICTED: uncharacterized protein LOC1059392...   

  NCBI.trasncript                                NCBI.product          gene  \
0  NW_012224770.1  